Código utilizado para análise de vibração mecânicas em uma barra tipo livre-engastada

Alunos:
Adler Armelini Furlan
Felipe Bendinelli Murça
Guilherme Siqueira de Aquino
Lucas Cocate Almeida
Veronica Maia Viana Marcial

Feito a partir do código do Professor, Luis Gustavo Giacon Villani, disponibilizado em aula.

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import scipy.signal as signal

In [6]:
#Parte para configurar os graficos 
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.serif'] = ['Times New Roman']
# pl.rcParams['font.size'] = 18
plt.rc('text', usetex=False)
plt.rcParams['figure.autolayout'] = True

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
np.random.seed(2021)

In [7]:
#Oscilador Linear 
# %% Integração usando método de Runge Kutta
def Linear_oscillator(M, C, K, F_matrix, t, disp0, vel0):
    # M: matriz  de massa do sistema
    # c: matriz de amortecimento do sistema
    # k: matriz de rigidez do sistema
    # t: vetor tempo
    # disp0: vetor deslocamento inicial
    # vel0: vetor velocidade inicial
    # F_matrix: matriz força aplicada
    M_inv = np.linalg.inv(M)
    disp = np.zeros((len(M), len(t)))
    vel = np.zeros((len(M), len(t)))
    acc = np.zeros((len(M), len(t)))
    disp[:, 0] = disp0
    vel[:, 0] = vel0
    acc[:, 0] = np.matmul(M_inv, (F_matrix[:, 0] - np.matmul(C, vel[:, 0]) - np.matmul(K, disp[:, 0])))
    dt = t[2] - t[1]
    for i in range(1, len(t)):
        k11 = dt * vel[:, i - 1]
        k12 = np.matmul(M_inv, dt * (F_matrix[:, i - 1] - np.matmul(C, vel[:, i - 1]) - np.matmul(K, disp[:, i - 1])))

        k21 = dt * (vel[:, i - 1] + k12 / 2)
        k22 = np.matmul(M_inv, dt * (
                    (F_matrix[:, i - 1] + F_matrix[:, i]) / 2 - np.matmul(C, (vel[:, i - 1] + k12 / 2)) - np.matmul(K, (
                        disp[:, i - 1] + k11 / 2))))

        k31 = dt * (vel[:, i - 1] + k22 / 2)
        k32 = np.matmul(M_inv, dt * (
                    (F_matrix[:, i - 1] + F_matrix[:, i]) / 2 - np.matmul(C, (vel[:, i - 1] + k22 / 2)) - np.matmul(K, (
                        disp[:, i - 1] + k21 / 2))))

        k41 = dt * (vel[:, i - 1] + k32)
        k42 = np.matmul(M_inv, dt * (
                    F_matrix[:, i] - np.matmul(C, (vel[:, i - 1] + k32)) - np.matmul(K, (disp[:, i - 1] + k31))))

        disp[:, i] = disp[:, i - 1] + (1 / 6) * (k11 + 2 * k21 + 2 * k31 + k41)
        vel[:, i] = vel[:, i - 1] + (1 / 6) * (k12 + 2 * k22 + 2 * k32 + k42)
        acc[:, i] = np.matmul(M_inv, (F_matrix[:, i] - np.matmul(C, vel[:, i]) - np.matmul(K, disp[:, i])))
    return (acc, vel, disp)

In [8]:
"""
Caso a ser testado
Caso 1 - 5 graus de liberdade
Caso 2 - 10 graus de liberdade
Caso 3 - 20 graus de liberdade
Caso 4 - 40 graus de liberdade
"""
caso = 3 
"""
Caso uma força seja aplicada, 
insira 'y' e caso contrário 'n'
"""
forca_ap = 'n'
"""
Define o tipo de analise que se deseja fazer
Análise dos modos de vibrar = 'modal'
Análise da resposta sistema = 'response'
Função de resposta em frequência = 'FRF'
"""
tipo_analise = 'response'


In [9]:
#####
rho = 2720         # Massa específica no SI [kg/m^3]
w_viga = 0.0105    # Dimensões da seção transversal da viga
t_viga = 0.007     # Dimensões da seção tranversal da viga
E = 70*(10 ** 9)   # Módulo de elásticidade do material
A = w_viga*t_viga  # Área da seção transversal
L = 0.210          # Comprimento da viga


In [10]:
if caso == 1:
    gdls =  5       # Número de graus de liberdade para o caso 1

if caso == 2:
    gdls = 10      # Número de graus de liberdade para o caso 2

if caso == 3:
    gdls = 20      # Número de graus de liberdade para o caso 3

if caso == 4:      # Número de graus de liberdade para o caso 3
    gdls = 40

In [11]:
L_i = (L / gdls) # Dividindo a viga de acordo com os graus de liberdade

In [12]:
m_eq = (rho * w_viga * t_viga) * L_i # Massa calculada através dos parâmetros do sistema

In [13]:
k_eq = (E*A)/((L_i))

In [14]:
k_last = (E*A)/((L_i/2))

In [15]:
M = np.zeros((gdls, gdls))

In [16]:
# Preenche a matriz massa, o último elemento da diagonal tem metade do comprimento, logo metade da massa
for i in range(0, gdls):
    for j in range(0, gdls):
        if (i == j) & (i !=(gdls - 1)):
            M[i, j] = round(m_eq, 8)
        elif (i == j) & (i == (gdls - 1)):
            M[i,j] = round(m_eq/2, 8)
        else:
            M[i,j] = 0

In [17]:
# Preenche a matriz de rigidez
K = np.zeros((gdls, gdls))
for i in range(0, gdls):
    for j in range(0, gdls):
        if (i == j) and (i !=(gdls - 1)):
            K[i, j] = 2*k_eq
            K[(i + 1), j] = - k_eq
            K[i, j+1] = - k_eq
        elif (i == j) and (i == (gdls - 1)):
            K[i, j] = k_last
            K[(i - 1), j] = - k_last
            K[i, (j - 1)] = - k_last
            K[i - 1, j - 1] = k_last + k_eq

In [18]:
# Definição da matriz de amortecimento
C = np.zeros((len(M), len(M)), dtype=np.float64)
alpha = 0.00001 # Coeficientes da matriz proporcional
beta = 0.000000001
C = alpha*M + beta*K # Matriz proporcional

In [19]:
# Preenche a matriz de amortecimento
for i in range(0, gdls):
    for j in range(0, gdls):
        C[i, j] = round(C[i, j], 8)

In [20]:
# Definição de parâmetros
Fs =  32768*2*2 *8* caso # Taxa de amostragem
Np =  32768*2*2*2*caso # Número de pontos
t = np.linspace(0, Np / Fs, Np, endpoint=False)  # Vetor tempo
w = np.linspace(0, Fs / 2, int(Np / 2), endpoint=False)  # Vetor frequencia em Hz
f = np.linspace(0, Fs, Np, endpoint=False)  # Com parte simetrica

In [21]:
X0 = np.zeros((len(M),))  # condições iniciais deslocamento
V0 = np.zeros((len(M),))  # condições iniciais velocidade

In [22]:
# Preenche a matriz força de acordo com a condição de contorno
F_applied = np.zeros((len(M), len(t)))
if forca_ap == 'y':
    F0 = 500  # amplitude da força de impulso [N]
    w_hz = Fs / 5;  # frequência central do impulso quanto mais rápido o impacto, maior essa frequência, melhor o resultado
    T = 1 / w_hz;  # período central do impulso
    Aux = round(Np / (2 * np.max(t) / T))  # pega apenas meio período da senoide
    F_impulse = np.zeros((Np,))  # criar o vetor força apenas com 0
    F_impulse[0:int(Aux)] = F0 * np.sin(2 * np.pi * w_hz * t[0:int(Aux)])  # cria o pulso (meio período de senóide)
    F_applied[len(M) - 1, :] = F_impulse[:] # Insere o vetor de força na matriz de força aplicada

In [23]:
if forca_ap == 'n':
    X0[len(M) - 1,] = 0.0005 # condição inicial apenas na massa n

In [ ]:
if tipo_analise == 'FRF':
    H = np.zeros((len(M),len(M),len(w)),dtype = 'complex_') # FRF analítico para o caso sem força aplicada
    for n in range(0,len(w)):
        H[:,:,n] = np.linalg.inv(K - M*((2*np.pi*w[n])**2) + complex(0,1)*C*2*np.pi*w[n])
    # Pega os picos da FRF, que são as frequências naturais.
    peaks_aux, _ = signal.find_peaks(H[0, 0, :])
    #print(f'Frequências Naturais numéricas: {w[peaks_aux]} [primeiro elemento]')
    peaks_aux, _ = signal.find_peaks(H[len(M) - 1, len(M) - 1, :])
    #print(f'Frquências Naturais numéricas: {w[peaks_aux]},  [último elemento]')
    # Plota a figura para FRF
    plt.figure()
    plt.grid(b=None, which='major', axis='both')
    plt.semilogy(w, abs(H[0,  len(M) - 1, :]), 'r', label=f'FRF [{1},{len(M)}]')
    plt.semilogy(w, abs(H[len(M) - 1, len(M) - 1, :]), 'c', label=f'FRF [{len(M) },{len(M) }]')
    plt.grid(b=None, which='major', axis='both')
    plt.xlim(0, 45000)
    plt.xlabel('$\omega$ [Hz]')
    plt.ylabel('$H (\omega)$ [m/N]')
    plt.legend()
    plt.show()

In [ ]:
 if tipo_analise == 'modal' or tipo_analise == 'response':

    # ---------------- Análise Modal Analítica ----------------- #
    n_freq = np.array([1, 2, 3, 4]) # Cria o vetor para as quatro primeiras frequências
    c = np.sqrt(E / rho) # Constante utilizada na conta
    wn_an = ((2 * n_freq - 1) * np.pi * c) / (2 * L) # Frequências naturais via análitica
    print(f'Resultado analítico: {wn_an / (2 * np.pi)}')

    x_pontos = np.linspace(L_i, L, gdls) # Cria os pontos para utilizar na análise modal análitica

    modal = np.zeros((len(x_pontos), len(n_freq)))  # Cria uma matriz inicial

    for i in range(0, len(n_freq)):
        modal[:, i] = np.sin((2 * n_freq[i] - 1) * np.pi * x_pontos / (2 * L)) # Análise dos modos de vibrar análitica

    # Normalizando os modos
    for i in range(0, len(n_freq)):
        modal[:, i] = modal[:, i] / np.linalg.norm(modal[:, i])
    # -------------------------------------------------------------------------------- #


    # ------------------------------ Análise Modal ----------------------------------- #
    # Calcula a matriz M^(1/2)
    M_aux = np.linalg.inv(M ** (1 / 2))
    # Calcula a matriz K tiu
    K_tiu = np.dot(M_aux, np.dot(K, M_aux))
    # Resolve o problema de auto valor e auto vetor
    a, b = np.linalg.eig(K_tiu)
    # Calcula os modos as frequências naturais via análise modal
    wn_modal = np.sqrt(a)
    print(f'Frequências Naturais (análise modal): {wn_modal / 2 / np.pi}')
    # Modos de vibrar na coordenada modal
    v = np.zeros((len(M), len(M)))
    for m in range(0, len(M)):
        v[:, m] = b[:, m] / np.linalg.norm(b[:, m])  # normaliza
    # Modos de vibrar na coordenada x
    u = np.dot(M_aux, v)
    for m in range(0, len(M)):
        u[:, m] = u[:, m] / np.linalg.norm(u[:, m]) # normaliza
    # Matriz auxiliarem para as transformações de coordenadas
    P = v
    S = np.dot(M_aux, P)
    S_inv = np.dot(np.transpose(P), M ** (1 / 2))
    # Calcula as condições iniciais em coordenadas modais
    R0 = np.dot(S_inv, X0)  # deslocamentos iniciais
    Rdot0 = np.dot(S_inv, V0)  # velocidades iniciais

    zeta_i = alpha / (2 * wn_modal) + beta * (wn_modal) / 2 # Calcula o coeficiente de amortecimento
    # Para o caso, como o amortecimento é sempre muito pequeno, zeta_i é sempre entre 0 e 1, o que implica em
    # um sistema subamortecido. E as repostas das próximas etapas são coerentes.

    wd = wn_modal * np.sqrt(1 - zeta_i ** 2) # Frequência amortecida

    if tipo_analise == 'response':
        acc, vel, disp = Linear_oscillator(M, C, K, F_applied, t, X0, V0)  # integração baseada no método numérico

        if forca_ap == 'n':
            r = np.zeros((len(M), len(t)))  # cria o vetor de zeros
            for m in range(0, len(M)):
                r[m, :] = np.exp(-zeta_i[m] * wn_modal[m] * t) * (
                            R0[m] * np.cos(wd[m] * t) + ((Rdot0[m] + zeta_i[m] * R0[m] * wn_modal[m]) / wd[m]) * np.sin(
                        wd[m] * t))

        if forca_ap == 'y':
            r = np.zeros((len(M), len(t)), dtype=complex)  # cria o vetor de zeros

            r_freq = np.zeros((len(M), len(t)), dtype=complex)
            f_iw = np.zeros((len(M), len(t)), dtype=complex)

            B_aux = np.dot(np.identity(len(M)), F_applied)
            f_i = np.dot(np.dot(np.transpose(P), M_aux), B_aux)

            I = np.identity(len(M))
            C_r = np.diag(2 * zeta_i * wn_modal)

            K_r = np.diag(a)

            H = np.zeros((len(M), len(M), len(w)), dtype='complex_')  # FRF analítico para o caso sem força aplicada
            H_aux = np.zeros((len(M), len(M), 2 * len(w)), dtype='complex')

            H = np.zeros((len(M), len(M), len(w)), dtype='complex_')  # FRF analítico para o caso sem força aplicada
            for n in range(0, len(w)):
                H[:, :, n] = np.linalg.inv(K_r - I * ((2 * np.pi * w[n]) ** 2) + complex(0, 1) * C_r * 2 * np.pi * w[n])

            for m in range(0, len(M)):
                H_aux[m, m, :] = np.concatenate(
                    (H[m, m, :], H[m, m, :][Np // 2 - 1] * np.ones((1,)), np.conj(H[m, m, :][Np // 2:0:-1, ])))

                f_iw[m, :] = sc.fft.fft(f_i[m, :])
                r_freq[m, :] = np.multiply(H_aux[m, m, :], f_iw[m, :])
                r[m, :] = sc.fft.ifft(r_freq[m, :])

        x_resp = np.dot(S, r)

        plt.figure()
        plt.grid(b=None, which='major', axis='both')
        plt.plot(t, disp[3, :], '-b', label='Resposta numérica')
        plt.plot(t, x_resp[3, :], '--k', label='Resposta modal')
        plt.xlabel('$t$ [s]')
        plt.ylabel(f'$x_{1}(t)$ [m]')
        plt.xlim(0, np.max(t))
        # plt.ylim(-1*10**(-5), 1*10**(-5))
        plt.legend()
        plt.show()

    else:
        massas = np.linspace(1, gdls, gdls)
        mass_aux = np.linspace(2, gdls, 20)
        for i in range(0, 4):
            plt.figure()
            plt.xlabel('Massas')
            plt.plot(massas, modal[:, i], '-ob', label='Analítico')
            plt.plot(massas, u[:, i], '-or', label='Modal Numérico')
            horiz_line_data = np.linspace(1, gdls, gdls)
            plt.ylim(-0.8, 0.8)
            plt.axhline(y=0, color='k', linestyle='--')
            plt.legend()
            plt.xticks(mass_aux)
            pl.show()
    #--------------------------------------------------------------------------------- #


Resultado analítico: [ 6039.28162138 18117.84486414 30196.4081069  42274.97134966]
Frequências Naturais (análise modal): [  6037.96184174  18082.12813332  30030.32919267  41816.53374165
  53372.39506422  64627.48921888  75510.11406696  85948.36866129
  95871.26239523 105209.69720767 113897.25768651 121870.80629977
 129070.91720509 135442.20674599 153263.96354122 151694.48643193
 149097.36523894 145498.97373904 140933.64576829 195622.68539344]
